# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Couchard Darius - Parent Paul - Donne Stefano

## Pump it Up: Data Mining the Water Table
####  April 29, 2021


# 1) Data Pre-Processing




In [13]:
require(tidyr)
require(dplyr)
require(plyr)
training_set<-read.csv("../Data/TrainingSet/4910797b-ee55-40a7-8668-10efd5c1b960.csv",header=TRUE) # loads the training set csv file (it's magic)
dim(training_set) # dimension of the set 
names(training_set) # names of the variables

traninin_labels<-read.csv("../Data/TrainingLabel/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv", header=TRUE) # Loads the corresponding labels


[1] 59400    40

[1] "id"                    "amount_tsh"            "date_recorded"        
 [4] "funder"                "gps_height"            "installer"            
 [7] "longitude"             "latitude"              "wpt_name"             
[10] "num_private"           "basin"                 "subvillage"           
[13] "region"                "region_code"           "district_code"        
[16] "lga"                   "ward"                  "population"           
[19] "public_meeting"        "recorded_by"           "scheme_management"    
[22] "scheme_name"           "permit"                "construction_year"    
[25] "extraction_type"       "extraction_type_group" "extraction_type_class"
[28] "management"            "management_group"      "payment"              
[31] "payment_type"          "water_quality"         "quality_group"        
[34] "quantity"              "quantity_group"        "source"               
[37] "source_type"           "source_class"          "waterpoint_type"      
[40] "waterpoint_type_group"

## How to enhance the data set :

After having analyzed the data set and assessed each variable relevance, it's needed to standardize the datas.

First, it's needed to remove empty values in the table (Imputation): each NaN or empty cell has to be removed or replaced, different solution exists : 
* Mapping NaN<-0 for nominal categorical variables
* Replace missing value by mean of column for numerical variables

Then, modifications have to be made depending on the nature of the data :
* If a column (variable) consists in continous numerical values : standardization is applied such has obtaining a new column with a mean value of 0 and a standard deviation of 1 (**longitude**)
* If a column is made of ordinal categorical variable (hierarchy between categories) : map each string to a numerical value (**water_quality**)
* In case of nominal categorical variable : apply one hot encoding -> create new column (with binary values) for each category (**source_type**)
<br/>


## Methods are implemented to apply these changes :

In [14]:
# METHOD TO REASSIGN EMPTY VALUES
NaN_handler_categorical <- function(column_name) { # input : column_name (name of the variable)
    training_set[training_set[,column_name] == "", ] <- 0 # select row where column element is empty string "" and assign to 0
    training_set[is.na(training_set[,column_name]),] <- 0 # select row where column element is NaN and assign to 0
    return(training_set)
}


# METHOD TO REASSIGN EMPTY VALUES
NaN_handler_num<- function(column_name) { # input : column_name (name of the variable)
    mean_col <- mean(training_set[,column_name], na.rm = TRUE)
    training_set[is.na(training_set[,column_name]),] <- mean_col
    return(training_set)
}



In [15]:
# STANDARDIZATION METHOD FOR VARIABLES WITH CONTINUOUS NUMERICAL VALUES
Standardization <- function(column_name){ # input : column_name (name of the variable)
    mean_col <- mean(training_set[,column_name], na.rm = TRUE) # mean of the variable
    sd_col <- sd(training_set[,column_name], na.rm = TRUE) # standard deviation of the variable
    training_set[,column_name]<-(training_set[,column_name]-mean_col)/sd_col # apply the transformation
    # now for the whole column : mean = 0 and sd = 1
    return(training_set)
}

In [16]:
# HANDLING OF NOMINAL CATEGORICAL VARIABLES
# before using : change notebook IOPub data rate limit with Jupyter  notebook --NotebookApp.iopub_data_rate_limit=jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
Nom_cat_handle <- function(column_name){
    training_set <- training_set %>% mutate(value = 1)  %>% spread(column_name, value,  fill = 0 )
    return(training_set)
}


## Process the data-set :

In [17]:
# COLUMNS TO DROP 
column_to_drop<-c("wpt_name","amount_tsh","date_recorded","gps_height","num_private","public_meeting","recorded_by",
                 "scheme_name","scheme_management","payment","quantity_group","source_class","subvillage","waterpoint_type",
                 "extraction_type","extraction_type_group","water_quality","source")
#TODO : adds others columns to drop
training_set<-training_set[,!(names(training_set) %in% column_to_drop)] # drop the desired column

In [18]:
# MANAGE SIMPLE NOMINAL CATEGORICAL VARIABLE
#TODO : apply one-hot encoding to other needed variables
training_set<-Nom_cat_handle("basin") # apply one-hot-encoding to the basin related column
training_set<-Nom_cat_handle("payment_type")
training_set<-Nom_cat_handle("quality_group")
training_set<-Nom_cat_handle("quantity")
training_set<-Nom_cat_handle("waterpoint_type_group")
training_set<-Nom_cat_handle("extraction_type_class")
training_set<-Nom_cat_handle("source_type")
# ...

In [19]:
# MANAGE ORDINAL CATEGORICAL VARIABLE

# variable PERMIT
# remap True : 1 , False : 0, Missing "" : NA
training_set$permit <- mapvalues(training_set$permit, 
          from=c("True","False",""), 
          to=c(1,0,NA))
training_set <- transform(training_set, permit = as.numeric(permit)) # transform column data type (char to int)
# replace missing value by mean of column
training_set<-NaN_handler_num("permit")
# Standardize Variable
training_set<-Standardization("permit")



In [20]:
# MANAGE POPULATION VARIABLE
# NaN have to be replaced by mean of region population
# column 1 = region code, column 2 = population mean in this region
region_code_frame <- data.frame("region" = unique(training_set$region_code),"mean_pop" = NA)
for(row in 1: nrow(region_code_frame) ){ 
    sel <-training_set[which(training_set[,"region_code"]==region_code_frame[row,1],),"population"] # select pop row with corresponding region
    region_code_frame[row,"mean_pop"]<-mean(sel[sel!=0],na.rm = TRUE)
}

# TODO : some region population mean are NaN , find a solution (same problem with region and district_code variables)
# temporairement on remplace les NaN par le mean des autres valeurs dans region_code_frame
region_code_frame[which(is.na(region_code_frame[,2])),2]<-mean(region_code_frame[,2],na.rm=TRUE)

# replace NaN value of population by mean region values
index<-which(training_set[,"population"]==0)
for(elem in index){
    training_set[elem,"population"]<-region_code_frame[training_set[elem,"region_code"],2] # replace NaN by their mean region value
}

# Now Standardize Population Variable 
training_set<-Standardization("population")


In [21]:
dim(training_set)
names(training_set)
training_set$population
training_set$permit

[1] 59400    56

[1] "id"                      "funder"                 
 [3] "installer"               "longitude"              
 [5] "latitude"                "region"                 
 [7] "region_code"             "district_code"          
 [9] "lga"                     "ward"                   
[11] "population"              "permit"                 
[13] "construction_year"       "management"             
[15] "management_group"        "Internal"               
[17] "Lake.Nyasa"              "Lake.Rukwa"             
[19] "Lake.Tanganyika"         "Lake.Victoria"          
[21] "Pangani"                 "Rufiji"                 
[23] "Ruvuma...Southern.Coast" "Wami...Ruvu"            
[25] "annually"                "monthly"                
[27] "never.pay"               "on.failure"             
[29] "per.bucket"              "colored"                
[31] "fluoride"                "good"                   
[33] "milky"                   "salty"                  
[35] "dry"                     "enough"                 
[37] "insufficient"            "seasonal"               
[39] "unknown"                 "cattle.trough"          
[41] "communal.standpipe"      "hand.pump"              
[43] "improved.spring"         "gravity"                
[45] "handpump"                "motorpump"              
[47] "rope.pump"               "submersible"            
[49] "wind.powered"            "borehole"               
[51] "dam"                     "other"                  
[53] "rainwater.harvesting"    "river.lake"             
[55] "shallow.well"            "spring"

[1] -0.353383894  0.033297333 -0.034541479 -0.468709874 -0.089199778
    [6] -0.597603617 -0.038285473 -0.038285473  0.543920191 -0.089199778
   [11]  0.180281425 -0.034541479 -0.038285473 -0.597603617 -0.038285473
   [16] -0.147606165 -0.520719630 -0.486800224  1.661428816 -0.597603617
   [21] -0.590819736 -0.038285473  0.191587894 -0.124993228 -0.247103089
   [26] -0.283283789 -0.011928542 -0.089199778 -0.597603617 -0.597603617
   [31] -0.326470280 -0.260670851 -0.192832040 -0.441574350 -0.373735538
   [36] -0.147606165  0.249383788 -0.326470280 -0.373735538 -0.079767353
   [41] -0.486800224 -0.372194595 -0.034541479 -0.598305636 -0.373735538
   [46] -0.038285473 -0.532026099 -0.598305636 -0.441574350 -0.554639036
   [51] -0.577251973 -0.147606165  0.249383788  0.088991810 -0.089199778
   [56] -0.597603617 -0.498106693 -0.260670851  0.431285029 -0.089199778
   [61] -0.147606165 -0.597603617  0.682288632 -0.305896726 -0.091073822
   [66] -0.598305636 -0.486800224 -0.089199778 -0.477755049 -0.597603617
   [71]  0.033297333  0.249383788  0.249383788 -0.147606165 -0.597603617
   [76] -0.089199778 -0.430267881 -0.283283789  1.435299443 -0.038285473
   [81] -0.372194595 -0.373735538  0.214200831 -0.038285473  0.630278877
   [86]  0.088991810  0.397365623 -0.089199778 -0.554639036 -0.326470280
   [91] -0.328509663  0.033297333 -0.509413161 -0.100118997 -0.089199778
   [96]  1.548364129  2.633785118  0.088991810 -0.052665160 -0.038285473
  [101] -0.351122601 -0.020973717 -0.260670851 -0.597603617 -0.430267881
  [106]  0.123749082 -0.038285473 -0.089199778 -0.597603617 -0.598305636
  [111] -0.328509663 -0.328509663 -0.089199778  0.249383788 -0.326470280
  [116] -0.089199778 -0.396348475  1.218215245 -0.598305636 -0.597603617
  [121] -0.372194595 -0.089199778 -0.089199778 -0.598305636 -0.598305636
  [126]  0.543920191 -0.598305636  0.089829676 -0.038285473  0.088991810
  [131] -0.089199778 -0.260670851  0.249383788 -0.038285473 -0.597603617
  [136] -0.486800224  0.249383788 -0.597603617 -0.089199778 -0.089199778
  [141]  0.249383788 -0.260670851  0.543920191 -0.597603617 -0.597603617
  [146] -0.260670851 -0.089199778 -0.509413161 -0.147606165  0.249383788
  [151]  3.018205051  0.249383788 -0.486800224  0.304652580  0.734298388
  [156] -0.597603617  0.249383788 -0.038285473 -0.038285473 -0.597603617
  [161] -0.034541479  0.417717266 -0.418961412  0.666459576  0.088991810
  [166]  5.053369405  0.543920191  1.774493502 -0.038285473 -0.518458336
  [171]  0.621233702  0.078523207 -0.572729386  0.417717266  0.829272725
  [176] -0.486800224 -0.045847948 -0.038285473  0.249383788  0.756911325
  [181] -0.597603617 -0.527503511 -0.328509663 -0.466448581 -0.079767353
  [186] -0.597603617  0.249383788 -0.089199778  0.249383788  0.249383788
  [191] -0.598305636 -0.372194595 -0.372194595 -0.486800224 -0.543332567
  [196] -0.260670851  0.530781953 -0.597603617 -0.373735538 -0.038285473
  [201] -0.089199778 -0.598305636 -0.509413161 -0.509413161 -0.089199778
  [206] -0.089199778  0.249383788  0.304652580 -0.373735538 -0.147606165
  [211] -0.554639036  0.249383788 -0.373735538  0.249383788  0.417717266
  [216]  0.249383788 -0.597603617 -0.038285473 -0.260670851  0.249383788
  [221] -0.598305636 -0.283283789 -0.396348475  0.689072514 -0.597603617
  [226] -0.597603617  0.983040698 -0.348861307 -0.038285473  0.088991810
  [231] -0.328509663  0.191587894 -0.598305636  0.824750137  0.010684396
  [236]  0.249383788  0.543920191 -0.597603617  0.249383788 -0.373735538
  [241] -0.034541479 -0.089199778 -0.326470280 -0.147606165 -0.597603617
  [246] -0.089199778 -0.597603617  1.661428816 -0.532026099  0.530781953
  [251] -0.475493755  3.244334424 -0.597603617  0.088991810 -0.372194595
  [256] -0.038285473  0.088991810  1.209170071 -0.597603617 -0.597603617
  [261] -0.089199778 -0.034541479 -0.326470280 -0.038285473 -0.326470280
  [266] -0.124993228  0.123749082 -0.598305636  0.249383788  5.121208217
  [271] -0.598305636  0.543920191  0.249383788 -0.038285473  0

[1] -1.5302158  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358
    [7]  0.6889358  0.6889358  0.6889358  0.6889358 -1.5302158 -1.5302158
   [13] -1.5302158  0.6889358  0.6889358 -1.5302158  0.6889358  0.6889358
   [19]  0.6889358  0.6889358 -1.5302158 -1.5302158  0.6889358  0.6889358
   [25]  0.6889358  0.6889358 -1.5302158  0.6889358  0.6889358  0.6889358
   [31]  0.6889358  0.6889358  0.6889358 -1.5302158 -1.5302158  0.6889358
   [37]  0.6889358 -1.5302158  0.6889358 -1.5302158  0.6889358 -1.5302158
   [43]  0.6889358  0.0000000  0.6889358  0.6889358  0.6889358  0.0000000
   [49]  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358
   [55]  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358
   [61] -1.5302158  0.6889358  0.6889358  0.6889358  0.6889358  0.0000000
   [67]  0.6889358  0.6889358  0.6889358 -1.5302158  0.6889358 -1.5302158
   [73]  0.6889358  0.6889358 -1.5302158 -1.5302158  0.6889358  0.6889358
   [79] -1.5302158  0.6889358 -1.5302158  0.6889358  0.6889358 -1.5302158
   [85] -1.5302158  0.6889358  0.6889358 -1.5302158  0.6889358 -1.5302158
   [91]  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358
   [97] -1.5302158  0.6889358  0.6889358 -1.5302158  0.6889358  0.6889358
  [103] -1.5302158 -1.5302158  0.6889358  0.6889358 -1.5302158  0.6889358
  [109]  0.6889358  0.0000000  0.6889358  0.6889358  0.6889358 -1.5302158
  [115]  0.6889358  0.6889358  0.6889358 -1.5302158  0.0000000  0.6889358
  [121] -1.5302158  0.6889358  0.6889358  0.0000000  0.0000000  0.6889358
  [127]  0.0000000  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358
  [133] -1.5302158  0.6889358  0.6889358 -1.5302158 -1.5302158 -1.5302158
  [139]  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358
  [145]  0.6889358 -1.5302158  0.6889358 -1.5302158  0.6889358  0.6889358
  [151]  0.6889358  0.6889358 -1.5302158  0.6889358 -1.5302158 -1.5302158
  [157]  0.6889358  0.6889358 -1.5302158 -1.5302158 -1.5302158 -1.5302158
  [163]  0.6889358  0.6889358  0.6889358 -1.5302158 -1.5302158  0.6889358
  [169] -1.5302158  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358
  [175]  0.6889358  0.6889358  0.6889358 -1.5302158 -1.5302158 -1.5302158
  [181]  0.6889358 -1.5302158  0.6889358 -1.5302158 -1.5302158  0.6889358
  [187]  0.6889358  0.6889358 -1.5302158  0.6889358  0.0000000  0.6889358
  [193]  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358
  [199]  0.6889358 -1.5302158 -1.5302158  0.0000000 -1.5302158 -1.5302158
  [205] -1.5302158 -1.5302158 -1.5302158 -1.5302158  0.6889358  0.6889358
  [211]  0.6889358  0.6889358  0.6889358 -1.5302158  0.6889358  0.6889358
  [217]  0.6889358  0.6889358  0.6889358 -1.5302158  0.0000000  0.6889358
  [223]  0.6889358  0.6889358  0.6889358 -1.5302158  0.6889358  0.6889358
  [229]  0.6889358  0.6889358  0.6889358  0.6889358  0.0000000 -1.5302158
  [235]  0.6889358  0.6889358 -1.5302158  0.6889358  0.6889358  0.6889358
  [241]  0.6889358  0.6889358 -1.5302158  0.6889358  0.6889358 -1.5302158
  [247]  0.6889358 -1.5302158  0.6889358 -1.5302158  0.6889358  0.6889358
  [253]  0.6889358  0.6889358 -1.5302158  0.6889358  0.6889358  0.6889358
  [259]  0.6889358 -1.5302158  0.6889358 -1.5302158 -1.5302158  0.6889358
  [265]  0.6889358  0.6889358  0.6889358  0.0000000  0.6889358  0.6889358
  [271]  0.0000000  0.6889358 -1.5302158  0.6889358 -1.5302158  0.6889358
  [277]  0.6889358  0.6889358 -1.5302158 -1.5302158  0.6889358 -1.5302158
  [283]  0.6889358  0.6889358  0.6889358  0.6889358  0.6889358 -1.5302158
  [289]  0.6889358 -1.5302158  0.6889358  0.6889358  0.6889358  0.6889358
  [295] -1.5302158 -1.5302158  0.6889358  0.6889358 -1.5302158  0.6889358
  [301] -1.5302158  0.6889358  0.6889358  0.6889358 -1.5302158  0.6889358
  [307]  0.6889358  0.0000000  0.6889358 -1.5302158  0.6889358  0.6889358
  [313]  0.6889358  0.6889358 -1.5302158 -1.5302158 -1.5302158  0.6889358
  [319] -1.5302158  0.6889358 -1.5302158 -1.5302158  0.6889358 -1.5302158
  [325] 